Following lesson https://www.youtube.com/watch?v=PMUfQInqsqg&list=PLtFmtptP5fJpkawGIGDptJI0BgN0OrueF&index=90&t=3370s

In [80]:
import tweepy as tw
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [ ]:
!pip install pyLDAvis

In [81]:
from __future__ import print_function

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

import spacy

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem.lancaster import LancasterStemmer

import operator

np.random.seed(2018)
import sys
import nltk
import csv, os

import pyLDAvis
import pyLDAvis.gensim_models

In [82]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [83]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [97]:
df = pd.read_csv(r'tesco_club.csv', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [98]:
df.head()

,tweet_location,user_screen,tweet
0,NaN,Camosocks,I wanna reiterate my theory that if you don't ...
1,UK,DanceHayday,"Yup, they even sent it to my 14yo daughter, ..."
2,NaN,shaun_mulqueen,I’ve made a decision and so should you all I ...
3,"Aberdeen, Scotland.",_boyMoon,"Medals for Mario Kart, Tesco Club Card, Reade..."
4,"my world, looking out",Myboysmum,Through no fault of my own I had to go into a ...


In [100]:
df['data'] = df.tweet.tolist()

In [ ]:
data = df.tweet.tolist()

df['data'] = [re.sub('\s*@\s*\s?', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('\?', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('\_', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('@"[\d-]"', '', str(sent)) for sent in df['data']]

#remove new lines characters?
df['data'] = [re.sub('\s+', '', str(sent)) for sent in df['data']]

df['data'] = df['data'].str.lower

In [105]:
def sent_to_words(sentences):
  for sentence in sentences :
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:3])

[['wanna', 'reiterate', 'my', 'theory', 'that', 'if', 'you', 'don', 'have', 'club', 'card', 'tesco', 'absolutely', 'hate', 'your', 'guts'], ['yup', 'they', 'even', 'sent', 'it', 'to', 'my', 'yo', 'daughter', 'although', 'she', 'does', 'have', 'club', 'card', 'tbf', 'just', 'seems', 'bit', 'capitalist', 'crass', 'to', 'me'], ['ve', 'made', 'decision', 'and', 'so', 'should', 'you', 'all', 'will', 'never', 'set', 'foot', 'in', 'tesco', 'shop', 'again', 'until', 'offered', 'the', 'same', 'price', 'on', 'products', 'as', 'your', 'club', 'card', 'holders']]


In [108]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[2]]])

['ve', 'made', 'decision', 'and', 'so', 'should', 'you', 'all', 'will', 'never', 'set', 'foot', 'in', 'tesco', 'shop', 'again', 'until', 'offered', 'the', 'same', 'price', 'on', 'products', 'as', 'your', 'club', 'card', 'holders']


In [119]:
def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
  return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
  return [trigram_mod[bigram_mod[doc]] for doc in texts]

#deleted lemmatization step from https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [125]:
#added lemmatization step from https://www.youtube.com/watch?v=TKjjlp5_r7o

def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[:3])

['reiterate theory club card absolutely hate gut', 'even send 14yo daughter club card tbf just seem bit capitalist crass 🙄', '’ve make decision so never set foot shop again ’m offer same price product club card holder']


In [143]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0:2])


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(1, 1), (2, 1), (6, 1), (8, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1)]]


In [149]:
word = id2word[1]
print (word)

card


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=34,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [151]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(14, '0.001*"forever" + 0.001*"frankly" + 0.001*"till" + 0.001*"wasting" + 0.001*"book" + 0.001*"promotion" + 0.001*"range" + 0.001*"shame" + 0.001*"bach" + 0.001*"environment"'), (33, '0.001*"forever" + 0.001*"frankly" + 0.001*"till" + 0.001*"wasting" + 0.001*"book" + 0.001*"promotion" + 0.001*"range" + 0.001*"shame" + 0.001*"bach" + 0.001*"environment"'), (20, '0.001*"forever" + 0.001*"frankly" + 0.001*"till" + 0.001*"wasting" + 0.001*"book" + 0.001*"promotion" + 0.001*"range" + 0.001*"shame" + 0.001*"bach" + 0.001*"environment"'), (29, '0.001*"forever" + 0.001*"frankly" + 0.001*"till" + 0.001*"wasting" + 0.001*"book" + 0.001*"promotion" + 0.001*"range" + 0.001*"shame" + 0.001*"bach" + 0.001*"environment"'), (6, '0.001*"forever" + 0.001*"frankly" + 0.001*"till" + 0.001*"wasting" + 0.001*"book" + 0.001*"promotion" + 0.001*"range" + 0.001*"shame" + 0.001*"bach" + 0.001*"environment"'), (21, '0.002*"have" + 0.002*"your" + 0.002*"to" + 0.002*"pay" + 0.002*"better" + 0.002*"sell" + 0.001

In [157]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=15)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
27    -0.244237 -0.030851       1        1  9.108549
31    -0.003492  0.250327       2        1  8.080559
1     -0.187871  0.188823       3        1  7.401174
11     0.095484  0.245292       4        1  7.138654
24    -0.241297 -0.134387       5        1  7.077353
19    -0.252829  0.082575       6        1  6.710600
30    -0.234289  0.012198       7        1  6.386966
12    -0.109258  0.222716       8        1  4.679989
4     -0.035000  0.239997       9        1  4.536726
13    -0.165645  0.100174      10        1  3.608550
5     -0.078873  0.140833      11        1  3.562019
26    -0.113139 -0.188977      12        1  3.476488
0      0.185162  0.171563      13        1  3.146277
7     -0.148169 -0.161090      14        1  2.907675
32    -0.026268 -0.265556      15        1  2.791013
17    -0.109327  0.052601      16        1  2.693197
15    -0.142714 -0.061985      17        1  2.689863
23     0.084351  0.155975      18        1  2.529376
25     0.159618 -0.229041      19        1  2.116235
18     0.012528 -0.199779      20        1  1.990027
16     0.181781  0.076118      21        1  1.516002
10    -0.031918 -0.064685      22        1  1.391769
3      0.007336 -0.012179      23        1  1.136203
22     0.069819  0.049893      24        1  1.042225
28     0.243527 -0.062850      25        1  0.838562
8      0.181935 -0.127666      26        1  0.833774
2      0.112715 -0.056055      27        1  0.077328
9      0.112091 -0.055901      28        1  0.076642
21     0.112348 -0.056077      29        1  0.076632
20     0.113126 -0.056401      30        1  0.075915
14     0.113126 -0.056401      31        1  0.075915
29     0.113126 -0.056401      32        1  0.075915
6      0.113126 -0.056401      33        1  0.075915
33     0.113126 -0.056401      34        1  0.075915, topic_info=      Term       Freq      Total Category  logprob  loglift
53     the  54.000000  54.000000  Default  15.0000  15.0000
31      to  52.000000  52.000000  Default  14.0000  14.0000
6     have  21.000000  21.000000  Default  13.0000  13.0000
14     you  30.000000  30.000000  Default  12.0000  12.0000
45      on  20.000000  20.000000  Default  11.0000  11.0000
..     ...        ...        ...      ...      ...      ...
10   tesco   0.002202  60.180697  Topic34  -6.5889  -3.0323
11    that   0.002202  14.817060  Topic34  -6.5889  -1.6307
12  theory   0.002202   1.349294  Topic34  -6.5889   0.7655
13   wanna   0.002202   1.349294  Topic34  -6.5889   0.7655
14     you   0.002202  30.852896  Topic34  -6.5889  -2.3642

[976 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
650      17  0.768836        able
66       10  0.371370       about
66       20  0.371370       about
0        10  0.741129  absolutely
436       5  0.690221     account
...     ...       ...         ...
15       17  0.084326        your
15       20  0.084326        your
15       23  0.084326        your
15       24  0.084326        your
33       11  0.742335         yup

[1055 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 32, 2, 12, 25, 20, 31, 13, 5, 14, 6, 27, 1, 8, 33, 18, 16, 24, 26, 19, 17, 11, 4, 23, 29, 9, 3, 10, 22, 21, 15, 30, 7, 34])